In [1]:
import sys
#sys.path.insert(0, "/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")
#sys.path.remove("/work/pi_wenlongzhao_umass_edu/27/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")

#print(sys.path)


In [2]:
if 'coir' in sys.modules:
    del sys.modules['coir']

In [3]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
import pandas as pd

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_data(dataset_name):
    tasks = get_tasks(tasks=[dataset_name])
    return tasks

In [5]:
def add_expl(tasks, dataset_name, explanation_df_path, col_name='explanation_deepseek_1_cleaned'):
    corpus, queries, qrels = tasks[dataset_name]
    expl_df = pd.read_csv(explanation_df_path)
    expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
    for _, row in expl_df.iterrows():
        corpus_id = row['corpus-id']
        explanation = row[col_name]

        if corpus_id in corpus and explanation and explanation.strip():
            corpus[corpus_id]['text'] = explanation  

    corpus = {doc_id: doc for doc_id, doc in corpus.items() if 'text' in doc and doc['text'].strip()}
    print(corpus['c1'])
    tasks[dataset_name] = (corpus, queries, qrels)

    print(f"Total docs in corpus after replacement: {len(tasks[dataset_name][0])}")
    missing_text = sum(1 for doc in tasks[dataset_name][0].values() if not doc.get('text'))
    print(f"Number of docs still missing 'text': {missing_text}")

In [6]:
def add_expl_2queries(tasks, dataset_name, explanation_df_path, col_name='explanation_deepseek_1_cleaned'):
    corpus, queries, qrels = tasks[dataset_name]
    expl_df = pd.read_csv(explanation_df_path)
    expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
    for _, row in expl_df.iterrows():
        query_id = row['query-id']
        explanation = row[col_name]

        if query_id in queries and explanation and explanation.strip():
            queries[query_id] = explanation  

    queries = {doc_id: doc for doc_id, doc in queries.items()}
    print(queries['q265734'])
    tasks[dataset_name] = (corpus, queries, qrels)

    print(f"Total docs in q after replacement: {len(tasks[dataset_name][1])}")

In [7]:
def run(model_name, tasks, llm_name, retrieval_name, dataset_name):
    model = YourCustomDEModel(model_name=model_name)

    evaluation = COIR(tasks=tasks,batch_size=256)
    if(retrieval_name == "bm25"):
        results = evaluation.run(model, output_folder=f"results/{dataset_name}/{llm_name}/{retrieval_name}")
    else:
        results = evaluation.run(model, output_folder=f"results/{dataset_name}/{llm_name}/{retrieval_name}/{model_name}")
    print(results)

In [8]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
csn_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_deepseek_allSplits.csv'

In [9]:
dataset_name = "CodeSearchNet-python"

In [10]:
tasks = load_data(dataset_name)

in tasks 
fetched data from hf


Casting the dataset: 100%|██████████| 280652/280652 [00:00<00:00, 3298400.61 examples/s]


data loader init


280310it [00:02, 116825.56it/s]


In [11]:
add_expl_2queries(tasks, "CodeSearchNet-python", csn_expl_path)

The docstring and code snippets represent a function that parses an XML string and extracts URLs from it. The function processes each element of the XML to collect URL data.
Total docs in q after replacement: 14918


In [12]:
run("intfloat/e5-base-v2", tasks, "deepseek1", "bm25", "codesearchnet")

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

Starting BM25-only search...
Initializing BM25...
Retrieving top-K documents using BM25...
Query q265734: Retrieved 1000 documents.
BM25 score distribution (min: 0.0, max: 41.58905571053599, mean: 10.0346)
Query q265735: Retrieved 1000 documents.
BM25 score distribution (min: 0.0, max: 98.73523335333331, mean: 22.4638)
Query q265736: Retrieved 1000 documents.
BM25 score distribution (min: 0.0, max: 155.80575532190124, mean: 47.1990)
Query q265737: Retrieved 1000 documents.
BM25 score distribution (min: 0.0, max: 91.74570599820379, mean: 13.5905)
Query q265738: Retrieved 1000 documents.
BM25 score distribution (min: 0.0, max: 87.45081336335085, mean: 22.3966)
Query q265739: Retrieved 1000 documents.
BM25 score distribution (min: 0.0, max: 256.9324977010869, mean: 47.6573)
Query q265740: Retrieved 1000 documents.
BM25 score distribution (m

KeyboardInterrupt: 

In [ ]:
# tasks = load_data("CodeSearchNet-python")
# run("BAAI/bge-base-en", tasks, "baseline", "dres", "codesearchnet")